# Backtest Example

In this notebook i will demonstrate a simple backtesting strategy over the QQQ etf

Buying criteria:
- 10 ema and 21 ema sloping upward
- 10 ema above 21 ema

Selling criteria:
- 10 ema and 21 ema sloping down
- 10 ema below 21 ema

In [1]:

from datetime import datetime
import backtrader as bt
from backtrader.analyzers import AnnualReturn, TradeAnalyzer
import yfinance as yf

from stats.tradeLog import TradeLog
from stats.strategySummery import build_summary_report
from stats.exporter import export_csv
class SmaCross(bt.Strategy):
    params = dict(
        period_fast=10,
        period_slow=21
    )

    def __init__(self):
        self.sma10 = bt.ind.SMA(period=self.p.period_fast)
        self.sma21 = bt.ind.SMA(period=self.p.period_slow)

    def next(self):
        sma10_above_sma21 = self.sma10[0] > self.sma21[0]
        sma10_sloping_up = self.sma10[0] > self.sma10[-1] if self.sma10[-1] else False
        sma21_sloping_up = self.sma21[0] > self.sma21[-1] if self.sma21[-1] else False

        if not self.position:
            if sma10_above_sma21 and sma10_sloping_up and sma21_sloping_up:
                self.buy()

        elif not sma10_above_sma21 and not sma10_sloping_up and not sma21_sloping_up:
            self.close()

cerebro = bt.Cerebro()
cerebro.addsizer(bt.sizers.AllInSizer)
cerebro.broker.setcash(100000)

qqq = yf.download("QQQ",  start=datetime(2017, 1, 1),end=datetime(2024, 12, 31))
qqq.columns = ["Open", "High", "Low", "Close", "Volume"] # Compatability with backtrader pandasData format
qqq_datafeed = bt.feeds.PandasData(dataname=qqq)
cerebro.adddata(qqq_datafeed, name="QQQ")

cerebro.addstrategy(SmaCross)
cerebro.addanalyzer(TradeLog, _name='trade_log')

cerebro.addanalyzer(AnnualReturn)
cerebro.addanalyzer(TradeAnalyzer)

sma_cross_strategy = cerebro.run()[0]

trade_table = sma_cross_strategy.analyzers.trade_log.get_analysis()
strategy_name = sma_cross_strategy.__class__.__name__
report = build_summary_report(sma_cross_strategy)
log_path = export_csv(trade_table, strategy_name)
summary_csv_path = export_csv(report, strategy_name)



cerebro.plot()
sma_cross_strategy.analyzers[0].get_analysis()


/var/folders/g8/lzy7w86j4w393km21jt_s8400000gn/T/ipykernel_13234/2401306235.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  qqq = yf.download("QQQ",  start=datetime(2017, 1, 1),end=datetime(2024, 12, 31))
[*********************100%***********************]  1 of 1 completed


20251127_2008
logs/trAade_log_SmaCross_20251127_2008.csv
reports/summary_SmaCross_20251127_2008.csv


<IPython.core.display.Javascript object>

[{'trade_id': 2,
  'data_name': 'QQQ',
  'side': 'short',
  'entry_time': '2017-06-29T00:00:00',
  'exit_time': '2017-06-29T00:00:00',
  'entry_price': 122.42623901367188,
  'exit_price': 131.93775720192525,
  'size': 0.0,
  'position_value_at_entry': 0.0,
  'position_fraction_of_equity': 0.0,
  'pnl_abs': 6323.7988136284375,
  'pnl_pct': 0.0,
  'holding_period_days': 0.0},
 {'trade_id': 4,
  'data_name': 'QQQ',
  'side': 'short',
  'entry_time': '2017-08-21T00:00:00',
  'exit_time': '2017-08-21T00:00:00',
  'entry_price': 136.3564910888672,
  'exit_price': 133.62195712295386,
  'size': 0.0,
  'position_value_at_entry': 0.0,
  'position_fraction_of_equity': 0.0,
  'pnl_abs': -2253.918919539707,
  'pnl_pct': 0.0,
  'holding_period_days': 0.0},
 {'trade_id': 6,
  'data_name': 'QQQ',
  'side': 'short',
  'entry_time': '2017-10-04T00:00:00',
  'exit_time': '2017-10-04T00:00:00',
  'entry_price': 137.32164001464844,
  'exit_price': 138.07736072674442,
  'size': 0.0,
  'position_value_at_ent